# OpenAI Function Calling
## Introduction

- Now that we can structure the output of llm models, we can go one step further.
- We will provide the llm a set of functions and have it suggest which function to call. 
- This concept is called `function calling`
- OpenAI has trained its model specifically for this, but this can be implemented by all models.

## Installation

In [1]:
%pip install -q openai


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## The function to call

Now assume that we want the model to call the following function to get the actual delivery date.

In [25]:
# This is the function that we want the model to be able to call
def get_delivery_date(order_id: str) -> str:
    # Connect to the database
    #conn = sqlite3.connect('ecommerce.db')
    #cursor = conn.cursor()
    return "01/01/2022"

We pass this function in the call and specify the description and the schema of the function.

## Marking it as a tool

In [26]:
import openai
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_delivery_date",
            "description": "Get the delivery date for a customer's order. Call this whenever you need to know the delivery date, for example when a customer asks 'Where is my package'",
            "parameters": {
                "type": "object",
                "properties": {
                    "order_id": {
                        "type": "string",
                        "description": "The customer's order ID.",
                    },
                },
                "required": ["order_id"],
                "additionalProperties": False,
            },
        }
    }
]


## Asking for action

Now consider the following chat between a customer and a chatbot.
It will ask for the order id. Ideally the llm will not just make things up , but will prompt to use the function to get the information.

In [34]:
messages = []
messages.append({"role": "system", "content": "You are a helpful customer support assistant. Use the supplied tools to assist the user."})
messages.append({"role": "user", "content": "Hi, can you tell me the delivery date for my order?"})
messages.append({"role": "assistant", "content": "Hi there! I can help with that. Can you please provide your order ID?"})
messages.append({"role": "user", "content": "i think it is order_12345"})

response = openai.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    tools=tools,
)

print(response)
assistant_message = response.choices[0].message




ChatCompletion(id='chatcmpl-BAf2ivUWlTIXA1SgLgOB99pupYPXQ', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_freLTfjy3oFmL8IXRcFHDpNO', function=Function(arguments='{"order_id":"order_12345"}', name='get_delivery_date'), type='function')], annotations=[]))], created=1741880748, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_06737a9306', usage=CompletionUsage(completion_tokens=20, prompt_tokens=140, total_tokens=160, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))


- We look at the results of the reply.
- Extract the arguments for get_delivery_date
- Note this code assumes we have already determined that the model generated a function call. See below for a more production ready example that shows how to check if the model generated a function call

In [35]:
import json
tool_call = response.choices[0].message.tool_calls[0]
print(tool_call)

ChatCompletionMessageToolCall(id='call_freLTfjy3oFmL8IXRcFHDpNO', function=Function(arguments='{"order_id":"order_12345"}', name='get_delivery_date'), type='function')


We can further extract the arguments that were passed

In [36]:
arguments = json.loads(tool_call.function.arguments)

order_id = arguments.get('order_id')
print(order_id)

# Call the get_delivery_date function with the extracted order_id
#delivery_date = get_delivery_date(order_id)

order_12345


All models can do function calling, but some have been specifically trained for it
<https://medium.com/@rushing_andrei/function-calling-with-open-source-llms-594aa5b3a304>

You need to extract the function and 

In [37]:
# Check if the model wants to call a tool
if assistant_message.tool_calls:
    # The model wants to use your get_delivery_date function
    for tool_call in assistant_message.tool_calls:
        if tool_call.function.name == "get_delivery_date":
            # Parse the arguments
            function_args = json.loads(tool_call.function.arguments)
            order_id = function_args.get("order_id")
            
            # Execute your function
            delivery_date = get_delivery_date(order_id)
            
            # Add the function response to messages
            messages.append(assistant_message)
            messages.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "content": delivery_date
            })
    
    # Get a new response from the model with the function result
    second_response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
    )
    
    print("🔹 Delivery Date:", second_response.choices[0].message.content)
else:
    # No tool call, just regular content
    print("🔹 Delivery Date:", assistant_message.content)

🔹 Delivery Date: The delivery date for your order (order_12345) is January 1, 2022. If you have any other questions or need further assistance, feel free to ask!
